In [1]:
import pickle
import pandas as pd
import numpy as np  
from tensorflow.keras.models import load_model
import tensorflow as tf

In [3]:
model=load_model('churn_model.h5')

In [5]:
with open('label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)
with open('one_hot_encoder.pkl', 'rb') as f:
    one_hot_encoder = pickle.load(f)
with open('scalar.pkl', 'rb') as f:
    scaler = pickle.load(f)


In [9]:
input_data = globals().get('input_data', {
	'creditScore': 600,
	'geography': 'France',
	'gender': 'Male',
	'age': 40,
	'tenure': 3,
	'balance': 60000,
	'numOfProducts': 2,
	'hasCrCard': 1,
	'isActiveMember': 1,
	'estimatedSalary': 50000
})

geo_input_df = pd.DataFrame([[input_data['geography']]], columns=['Geography'])
geo_encoded = one_hot_encoder.transform(geo_input_df)
geo_encoded_df = pd.DataFrame(
	geo_encoded,
	columns=one_hot_encoder.get_feature_names_out(['Geography'])
)


In [11]:
input_df = pd.DataFrame([input_data])

geo_col = 'Geography' if 'Geography' in input_df.columns else 'geography'
input_df = pd.concat([input_df.drop(columns=[geo_col]), geo_encoded_df], axis=1)

In [12]:
input_df

,creditScore,gender,age,tenure,balance,numOfProducts,hasCrCard,isActiveMember,estimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [15]:
# Match training-time column names
rename_map = {
	'creditScore': 'CreditScore',
	'gender': 'Gender',
	'age': 'Age',
	'tenure': 'Tenure',
	'balance': 'Balance',
	'numOfProducts': 'NumOfProducts',
	'hasCrCard': 'HasCrCard',
	'isActiveMember': 'IsActiveMember',
	'estimatedSalary': 'EstimatedSalary'
}
input_df = input_df.rename(columns=rename_map)

# Encode Gender like training
if 'Gender' in input_df.columns and input_df['Gender'].dtype == 'object':
	input_df['Gender'] = label_encoder.transform(input_df['Gender'])

# Align exactly to scaler training columns (names + order)
if hasattr(scaler, 'feature_names_in_'):
	for col in scaler.feature_names_in_:
		if col not in input_df.columns:
			input_df[col] = 0
	input_df = input_df[scaler.feature_names_in_]

input_scaled = scaler.transform(input_df)

In [16]:
prediction=model.predict(input_scaled)
print("Churn Probability:", prediction[0][0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 663ms/step
Churn Probability: 0.03114957
